# Using healpy and yt to make Mollweide projections of spheres

In [1]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import nugridpy.utils as utils
from ppmpy import ppm
import os
import yt

dir_repo = '/data/ASDR'
dir_project = 'H-core-M25'
# run, dump = 'M29-768', 650
run, dump = 'M35-1536', 425

path = os.path.join(dir_repo,dir_project,run)
moms = ppm.MomsDataSet(os.path.join(path,'myavsbq'),dump)

x,y,z,r = moms.get_grid()

res = moms.momsdata.resolution    # should be 192
vor_matrix = np.reshape(moms.get(6,dump),(res,res,res))

data = dict(vorticity=vor_matrix)
bbox = np.array([[min(x),max(x)], [min(y),max(y)], [min(z),max(z)]])
ds = yt.load_uniform_grid(data=data, domain_dimensions=[res,res,res], length_unit='Mm', bbox=bbox)

249 .aaa files found in '/data/ASDR/H-core-M25/M35-1536/myavsbq/.
Dump numbers range from 3 to 437.
The PPMstar grid is being constructed, this can take a moment


yt : [INFO     ] 2018-12-01 20:12:35,818 Parameters: current_time              = 0.0
yt : [INFO     ] 2018-12-01 20:12:35,819 Parameters: domain_dimensions         = [384 384 384]
yt : [INFO     ] 2018-12-01 20:12:35,821 Parameters: domain_left_edge          = [-2493.48950195 -2493.48950195 -2493.48950195]
yt : [INFO     ] 2018-12-01 20:12:35,823 Parameters: domain_right_edge         = [2493.48950195 2493.48950195 2493.48950195]
yt : [INFO     ] 2018-12-01 20:12:35,824 Parameters: cosmological_simulation   = 0.0


In [2]:
from yt.units import cm, Mm, m
radius = 1000. * Mm
all = ds.all_data()   # flat collection of all points
surface = ds.surface(data_source=all, surface_field=('index', 'radius'), field_value=radius)
numTriangles = surface.triangles.shape[0]
print(numTriangles, 'triangles on the surface')

223208 triangles on the surface


In [3]:
points = []
for i, t in enumerate(surface.triangles):
    points.append([t[:,0].mean(), t[:,1].mean(), t[:,2].mean()])

coords = np.array(points)
theta, phi = hp.vec2ang(coords)   # latitude,longitude of all trianles
theta -= np.pi/2.
phi -= np.pi

In [4]:
close(1);figure(1,figsize=(8,6))
ax = plt.subplot(111, projection = 'mollweide')
im = ax.tripcolor(phi,theta,surface['vorticity'])   # create a pseudocolor plot of an unstructured triangular grid
frame = plt.gca(projection='mollweide')
frame.axes.xaxis.set_ticklabels([])
frame.axes.yaxis.set_ticklabels([])
cbar = plt.colorbar(im, orientation="horizontal", pad=0.2, shrink=0.75)
cbar.set_label('vorticity')
plt.show()


FigureCanvasNbAgg()

yt : [INFO     ] 2018-12-01 20:19:48,185 Extracting (sampling: ('stream', 'vorticity'))
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fe5d7281940>
DEBUG:matplotlib.colorbar:Using auto colorbar locator on colorbar
DEBUG:matplotlib.colorbar:locator: <matplotlib.colorbar._ColorbarAutoLocator object at 0x7fe5d7281940>
DEBUG:matplotlib.colorbar:Setting pcolormesh


In [5]:
plt.savefig('vorticity-mollweide.png')

DEBUG:matplotlib.axes._base:update_title_pos
DEBUG:matplotlib.axes._base:update_title_pos
